In [ ]:
#step1 Download the dataset

In [ ]:
pip install split-folders

In [ ]:
#step2 Split the dataset into Training set and Test set
import splitfolders  # or import split_folders

# Split with a ratio.
# To only split into training and validation set, set a tuple to `ratio`, i.e, `(.8, .2)`.
splitfolders.ratio("flowers", output="Output", seed=1337, ratio=(.75, 0, .25), group_prefix=None) # default values


In [ ]:
pip install opencv-python

In [2]:
pip install numpy -U

Requirement already up-to-date: numpy in c:\users\s-n_e\appdata\roaming\python\python38\site-packages (1.19.4)
Note: you may need to restart the kernel to use updated packages.


ERROR: Error checking for conflicts.
Traceback (most recent call last):
  File "C:\Users\s-n_e\anaconda3\lib\site-packages\pip\_vendor\pkg_resources\__init__.py", line 3021, in _dep_map
    return self.__dep_map
  File "C:\Users\s-n_e\anaconda3\lib\site-packages\pip\_vendor\pkg_resources\__init__.py", line 2815, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\s-n_e\anaconda3\lib\site-packages\pip\_vendor\pkg_resources\__init__.py", line 3012, in _parsed_pkg_info
    return self._pkg_info
  File "C:\Users\s-n_e\anaconda3\lib\site-packages\pip\_vendor\pkg_resources\__init__.py", line 2815, in __getattr__
    raise AttributeError(attr)
AttributeError: _pkg_info

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\s-n_e\anaconda3\lib\site-packages\pip\_inter

In [1]:
pip install numpy==1.19.3

  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.4
    Uninstalling numpy-1.19.4:
      Successfully uninstalled numpy-1.19.4
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an EnvironmentError: [WinError 5] Access is denied: 'C:\\Users\\s-n_e\\AppData\\Roaming\\Python\\Python38\\site-packages\\~umpy\\.libs\\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll'
Consider using the `--user` option or check the permissions.



In [2]:
#Step3 Create Python program to read the Training set images and create a tabular dataset
import os
import cv2 

from skimage import feature
 
images_train = []
labels_train = []
images_test = []
labels_test = []
#lst = []
# get all the image folder paths 
image_paths = os.listdir('Output/train') 
for path in image_paths:
    # get all the image names
    all_images_train = os.listdir(f"Output/train/{path}")
    all_images_test = os.listdir(f"Output/test/{path}")

    # iterate over the image names, get the label
    for image_train in all_images_train:
        image_path_train = f"Output/train/{path}/{image_train}"
        image_train = cv2.imread(image_path_train)
        image_train = cv2.resize(image_train, (128, 256))

        # get the HOG descriptor for the image
        hog_desc_train = feature.hog(image_train, orientations=9, pixels_per_cell=(8, 8),
            cells_per_block=(2, 2), transform_sqrt=True, block_norm='L2-Hys')
    
        # update the data and labels
        images_train.append(hog_desc_train)
        labels_train.append(path) 
        
    # iterate over the image names, get the label
    for image_test in all_images_test:
        image_path_test = f"Output/test/{path}/{image_test}"
        image_test = cv2.imread(image_path_test)
        image_test = cv2.resize(image_test, (128, 256))

        # get the HOG descriptor for the image
        hog_desc_test = feature.hog(image_test, orientations=9, pixels_per_cell=(8, 8),
            cells_per_block=(2, 2), transform_sqrt=True, block_norm='L2-Hys')
    
        # update the data and labels
        images_test.append(hog_desc_test)
        labels_test.append(path) 
        
        """#add features and label in the same tabular dataset without split
        test01=hog_desc.tolist()
        test01.append(path) 
        lst.append(test01)"""
        


In [19]:
#to tabulate dataset
#pip install tabulate

#from tabulate import tabulate
#table =tabulate(lst)

Note: you may need to restart the kernel to use updated packages.


In [3]:
#Pandas DataFrame
import pandas as pd
df = pd.DataFrame(lst)
df.head()

NameError: name 'lst' is not defined

In [5]:
#Step4 Build and train a classifier model: naive_bayes 
from sklearn.naive_bayes import GaussianNB as gnb
# Creating the classifier object
model = gnb()
 
# Performing training
model.fit(images_train, labels_train) 
# Predicton on test 
y_pred = model.predict(images_test) 
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(labels_test, y_pred))

Accuracy: 0.68


In [85]:
import joblib
  
# Save the model as a pickle in a file 
joblib.dump(model, 'trained_model.pkl') 
  
# Load the model from the file 
naive_bayes_from_joblib = joblib.load('trained_model.pkl')  
  
# Use the loaded model to make predictions 
#naive_bayes_from_joblib.predict(images_test) 

In [6]:
# predict on the test images
print('Evaluating on test images...')
# loop over the test dataset folders
for (i, imagePath) in enumerate(os.listdir(f"test_images/flowers/")):
    image = cv2.imread(f"test_images/flowers/{imagePath}")
    resized_image = cv2.resize(image, (128, 256))
    # get the HOG descriptor for the test image
    (hog_desc, hog_image) = feature.hog(resized_image, orientations=9, pixels_per_cell=(8, 8),
        cells_per_block=(2, 2), transform_sqrt=True, block_norm='L2-Hys', visualize=True)
    # prediction
    pred = model.predict(hog_desc.reshape(1, -1))[0]
    # convert the HOG image to appropriate data type. We do...
    # ... this instead of rescaling the pixels from 0. to 255.
    hog_image = hog_image.astype('float64')
    # show thw HOG image
    cv2.imshow('HOG Image', hog_image)
    # put the predicted text on the test image
    cv2.putText(image, pred.title(), (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1.0,
        (0, 255, 0), 2) 
    cv2.imshow('Test Image', image)
    cv2.imwrite(f"outputs/flowers_hog_{i}.jpg", hog_image*255.) # multiply by 255. to bring to OpenCV pixel range
    cv2.imwrite(f"outputs/flowers_pred_{i}.jpg", image)
    cv2.waitKey(0)

Evaluating on test images...


In [1]:
pip install fastai

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement torchvision>=0.8 (from fastai) (from versions: 0.1.6, 0.1.7, 0.1.8, 0.1.9, 0.2.0, 0.2.1, 0.2.2, 0.2.2.post2, 0.2.2.post3, 0.5.0)
ERROR: No matching distribution found for torchvision>=0.8 (from fastai)


In [ ]:
conda install -c pytorch -c fastai fastai

In [ ]:
from flask import Flask, request, jsonify
import load_learner
import open_image
from flask_cors import CORS,cross_origin
app = Flask(__name__)
CORS(app, support_credentials=True)

# load the learner
learn = load_learner(path='./models', file='trained_model.pkl')
classes = learn.data.classes


def predict_single(img_file):
    'function to take image and return prediction'
    prediction = learn.predict(open_image(img_file))
    probs_list = prediction[2].numpy()
    return {
        'category': classes[prediction[1].item()],
        'probs': {c: round(float(probs_list[i]), 5) for (i, c) in enumerate(classes)}
    }


# route for prediction
@app.route('/predict', methods=['POST'])
def predict():
    return jsonify(predict_single(request.files['image']))

if __name__ == '__main__':
    app.run()